<a href="https://colab.research.google.com/github/AP-047/RClass-Classification-by-Rational-Approximation/blob/main/notebooks/milestone_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**1. Importing the MNIST dataset and selecting a subset**

In [19]:
import numpy as np
from keras.datasets import mnist

# Load the MNIST dataset
(X_full, y_full), (_, _) = mnist.load_data()

# Initialize containers for subsets
subset_X = []
subset_y = []

# Draw 100 images per digit (0-9)
for digit in range(10):
    digit_indices = np.where(y_full == digit)[0]
    selected_indices = digit_indices[:100]  # Select the first 100 images for this digit
    subset_X.append(X_full[selected_indices])
    subset_y.append(y_full[selected_indices])

# Combine the subsets
subset_X = np.vstack(subset_X)  # Stack all selected images
subset_y = np.hstack(subset_y)  # Stack all selected labels

# Normalize pixel values to [0, 1]
subset_X = subset_X.astype("float32") / 255.0

#____Testing____
print(f"Subset X shape (images): {subset_X.shape}")  # Expected shape: (1000, 28, 28)
print(f"Subset y shape (labels): {subset_y.shape}")  # Expected shape: (1000,)

Subset X shape (images): (1000, 28, 28)
Subset y shape (labels): (1000,)


**2. Dimensionality reduction using PCA**

In [96]:
from sklearn.decomposition import PCA

# Flatten the images to shape (1000, 784)
subset_X_flattened = subset_X.reshape(subset_X.shape[0], -1)

# Apply PCA to reduce to 50 components
pca = PCA(n_components=50)
subset_X_reduced = pca.fit_transform(subset_X_flattened)


#____Testing____
# Print the new shape of the dataset
print(f"Reduced X shape (after PCA): {subset_X_reduced.shape}")
# Print the explained variance ratio to verify the retained variance
explained_variance = np.sum(pca.explained_variance_ratio_)
print(f"Total explained variance retained: {explained_variance * 100:.2f}%")

#____Testing____
# Set the desired label (digit) to inspect
desired_label = 3
# Find the index of the first occurrence of the desired label
index = np.where(subset_y == desired_label)[0][0]
# Get the corresponding reduced vector from PCA
selected_vector = subset_X_reduced[index]
# Print the vector and its length
print(f"Vector for digit {desired_label}: {selected_vector}")
print(f"Length of the vector: {len(selected_vector)}")

Reduced X shape (after PCA): (1000, 50)
Total explained variance retained: 100.00%
Vector for digit 3: [ 2.17378044e+00  2.27193213e+00  5.60363650e-01  3.62105036e+00
 -1.00318216e-01  3.54436851e+00  1.42353189e+00  6.69775188e-01
 -9.06186737e-03  5.96990049e-01  1.07461810e-01  1.01029232e-01
 -7.79241979e-01 -6.47653267e-02  4.35082167e-01 -4.70671281e-02
 -1.33320749e+00  1.05579937e+00 -9.92273614e-02 -3.55659664e-01
 -1.46669865e-01  4.77436513e-01  2.06573680e-01 -1.34055531e+00
 -1.96844786e-01  6.70857608e-01  1.25434130e-01 -3.05720836e-01
 -6.78535581e-01 -1.37481436e-01  2.02038169e-01  1.97802827e-01
 -3.41805555e-02 -1.39783725e-01  4.61463928e-01  6.18020535e-01
  8.46668333e-02  2.88082391e-01  1.43132627e-03  4.39935595e-01
  6.20617867e-01  5.95056079e-02  1.78914532e-01 -7.14280307e-02
  7.75693133e-02 -5.21081779e-03  1.88026950e-01  3.14979181e-02
 -1.08453162e-01 -1.82146132e-01]
Length of the vector: 50


**3. Implement simple multi-variate rational function**

In [100]:
import numpy as np

# Define number of variables and the polynomial degree
num_variables = 50
degree = 5

# Randomly initialize coefficients
np.random.seed(47)  # for reproducibility
nm_cf = np.random.rand(num_variables + 1)  # Numerator coefficients
dn_cf = np.random.rand(num_variables + 1)  # Denominator coefficients

# Define the rational function
def rational_function(x, nm_cf, dn_cf):
    # Construct the numerator (P) and denominator (Q) polynomials
    P = sum(nm_cf[i] * x[i]**(max(degree - i, 1)) for i in range(len(x))) + nm_cf[-1]
    Q = sum(dn_cf[i] * x[i]**(max(degree - i, 1)) for i in range(len(x))) + dn_cf[-1]

    # Avoid division by zero
    if Q == 0:
        return float('inf')
    return P / Q

#____Testing____
sample_vector = subset_X_reduced[0]  # First PCA-reduced vector
output = rational_function(sample_vector, nm_cf, dn_cf)

print(f"Numerator coefficients (nm_cf): {nm_cf}")
print(f"Denominator coefficients (dn_cf): {dn_cf}")
print(f"Output of the rational function: {output}")

#____Testing____
# Function to print the polynomials
def print_polynomials(nm_cf, dn_cf, degree, num_variables):
    print("\nNumerator Polynomial (P):")
    poly_terms = [f"{nm_cf[i]:.2f} * x{i+1}^{max(degree - i, 1)}" for i in range(num_variables)]
    poly_terms.append(f"{nm_cf[-1]:.2f}")
    print(" + ".join(poly_terms))

    print("\nDenominator Polynomial (Q):")
    poly_terms = [f"{dn_cf[i]:.2f} * x{i+1}^{max(degree - i, 1)}" for i in range(num_variables)]
    poly_terms.append(f"{dn_cf[-1]:.2f}")
    print(" + ".join(poly_terms))

# Print polynomials for verification
print_polynomials(nm_cf, dn_cf, degree, num_variables)

Numerator coefficients (nm_cf): [0.11348847 0.97448309 0.72873463 0.35146781 0.70760514 0.7996046
 0.64556185 0.41459961 0.70603101 0.24664938 0.25599243 0.02401135
 0.09872595 0.30043644 0.64085568 0.32220795 0.18549414 0.91719355
 0.2709208  0.27354789 0.95441268 0.12711457 0.74726485 0.00523796
 0.85679061 0.6959562  0.5530257  0.9352358  0.51262353 0.17761207
 0.53686579 0.29345982 0.01060205 0.88380072 0.65641065 0.94225597
 0.7449453  0.26721045 0.36186288 0.52640389 0.54688497 0.25867032
 0.17463635 0.36071026 0.14018227 0.38907866 0.47188833 0.96882692
 0.14556745 0.51425472 0.52765183]
Denominator coefficients (dn_cf): [0.30524252 0.15967476 0.59691153 0.10372881 0.56796609 0.38630638
 0.08472205 0.56199918 0.64807168 0.66193395 0.18924281 0.95410895
 0.05902446 0.88020624 0.78375816 0.25247741 0.92709052 0.4447506
 0.37715133 0.89382348 0.75405387 0.77445802 0.87904851 0.48148232
 0.30290073 0.44292787 0.52813285 0.60814462 0.52524497 0.97796815
 0.60203988 0.83529596 0.08537

**Optimization**

**1. Initialization Phase**

In [98]:
import numpy as np

# Initialization
uL = 0  # Lower bound for deviation
uH = 20  # Initial upper bound
precision = 1e-5  # Precision for bisection

# total number of samples (images) in the subset of your PCA-reduced dataset
num_samples = len(subset_X)

x_in = subset_X_reduced  # input data - PCA-reduced vectors
y_targets = subset_y  # labels as the target values

#____Testing____
# Print initialization values for verification
print("Initialization Phase (Adapted):")
print(f"Lower bound (uL): {uL}")
print(f"Upper bound (uH): {uH}")
print(f"Precision: {precision}")
print(f"Number of samples: {num_samples}")
#print(f"First 5 targets: {y_targets[:5]}")
#print(f"First 5 input vectors: {x_in[:5]}")

Initialization Phase (Adapted):
Lower bound (uL): 0
Upper bound (uH): 20
Precision: 1e-05
Number of samples: 1000


**2. Bisection Loop**

In [99]:
from scipy.optimize import linprog

def check_feasibility(z, x_data, y_data, nm_cf, dn_cf, degree, uL, uH):
    num_coefficients = len(nm_cf) + len(dn_cf)  # Total number of coefficients

    # Construct A_ub and b_ub for all samples
    A = []
    b = []

    for i, (x, y) in enumerate(zip(x_data, y_data)):
        # Construct rows for the numerator and denominator
        numerator_row = [x[j] ** max(degree - j, 1) for j in range(len(x))] + [1]  # Add bias term
        denominator_row = [-1 * x[j] ** max(degree - j, 1) for j in range(len(x))] + [-1]  # Add bias term

        # Append rows to A and corresponding constraints to b
        A.append(numerator_row + denominator_row)  # Combine numerator and denominator rows
        b.append(z * sum(denominator_row) - y)  # Update the constraint value

    # Convert A and b to NumPy arrays
    A = np.array(A)
    b = np.array(b)

    # Debugging: Print the shape of A and length of b
    print(f"A_ub shape: {A.shape}")
    print(f"b_ub length: {len(b)}")
    print(f"Number of coefficients: {num_coefficients}")

    # Set bounds for the coefficients
    bounds = [(uL, uH) for _ in range(num_coefficients)]

    # Solve the linear programming problem
    result = linprog(
        c=[0] * num_coefficients,  # Objective function doesn't matter for feasibility
        A_ub=A,
        b_ub=b,
        bounds=bounds,
        method="highs",
    )

    # If infeasible, return False
    if result.status != 0:  # Check if optimization succeeded
        return False

    return True

def bisection_loop(x_data, y_data, nm_cf, dn_cf, degree, uL, uH, precision):
    """
    Perform the bisection loop to minimize the maximum deviation z.
    :param x_data: PCA-reduced input data.
    :param y_data: Target labels.
    :param nm_cf: Coefficients for numerator.
    :param dn_cf: Coefficients for denominator.
    :param degree: Degree of the polynomial.
    :param uL: Initial lower bound for deviation.
    :param uH: Initial upper bound for deviation.
    :param precision: Precision for stopping the loop.
    :return: Optimal deviation z and coefficients.
    """
    while uH - uL > precision:
        z = (uL + uH) / 2
        if check_feasibility(z, x_data, y_data, nm_cf, dn_cf, degree, uL, uH):
            uH = z
        else:
            uL = z
        print(f"Current bounds: uL = {uL}, uH = {uH}")

    return (uL + uH) / 2  # Return the final optimal deviation z

# Run the bisection loop
optimal_z = bisection_loop(subset_X_reduced, subset_y, nm_cf, dn_cf, degree, uL, uH, precision)
print(f"Optimal deviation (z): {optimal_z}")

A_ub shape: (1000, 102)
b_ub length: 1000
Number of coefficients: 102
Current bounds: uL = 0, uH = 10.0
A_ub shape: (1000, 102)
b_ub length: 1000
Number of coefficients: 102
Current bounds: uL = 0, uH = 5.0
A_ub shape: (1000, 102)
b_ub length: 1000
Number of coefficients: 102
Current bounds: uL = 0, uH = 2.5
A_ub shape: (1000, 102)
b_ub length: 1000
Number of coefficients: 102
Current bounds: uL = 1.25, uH = 2.5
A_ub shape: (1000, 102)
b_ub length: 1000
Number of coefficients: 102
Current bounds: uL = 1.875, uH = 2.5
A_ub shape: (1000, 102)
b_ub length: 1000
Number of coefficients: 102
Current bounds: uL = 2.1875, uH = 2.5
A_ub shape: (1000, 102)
b_ub length: 1000
Number of coefficients: 102
Current bounds: uL = 2.34375, uH = 2.5
A_ub shape: (1000, 102)
b_ub length: 1000
Number of coefficients: 102
Current bounds: uL = 2.421875, uH = 2.5
A_ub shape: (1000, 102)
b_ub length: 1000
Number of coefficients: 102
Current bounds: uL = 2.4609375, uH = 2.5
A_ub shape: (1000, 102)
b_ub length: 10

**3. Update Coefficients with Optimal z**